In [1]:
import os
import pandas as pd
import numpy as np
import random
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.applications import DenseNet201
from keras import Model

/home/csci8523/attan005/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
pretrained_model = DenseNet201(input_shape=(128,128,3),
                                            include_top=False,
                                            weights='imagenet',
                                            pooling='max')
pretrained_model.trainable = False
input_layer = pretrained_model.input
dropout_layer1 = Dropout(0.25)(pretrained_model.output)
dense_layer1 = Dense(512, activation='relu')(dropout_layer1)
dense_layer2 = Dense(128, activation='relu')(dense_layer1)
dropout_layer2 = Dropout(0.20)(dense_layer2)
output_layer = Dense(116, activation='softmax')(dropout_layer2)
model = Model(inputs=input_layer, outputs=output_layer)

In [3]:
model.load_weights("model_dense.h5")

In [4]:
valdf = pd.read_csv("val_clean_3.csv")
images_pixels = []
temp_labels = []
y_test_ids = []
for ind in valdf.index:
    img = valdf['images'][ind].split(" ")[:-1]
    for i in img:
        finalpath = "validation_images_model_3/"+i
        #print(finalpath)
        img_pix = cv2.imread(finalpath,1)
        images_pixels.append(cv2.resize(img_pix, (128,128)))
        temp_labels.append(str(valdf['landmark_id'][ind]-1))
        y_test_ids.append(i)
#print(y_test_ids)
shuf = list(zip(images_pixels,temp_labels, y_test_ids))
random.shuffle(shuf)

val_data, labels_data_val, y_test_ids = zip(*shuf)
print('Images: ', len(val_data))
print('Image labels: ', len(labels_data_val))
X_data_val = np.array(val_data) / 255
Y_data_val =  to_categorical(labels_data_val, num_classes = 116) 

Images:  3690
Image labels:  3690


In [5]:
import csv
y_pred = np.argmax(model.predict(X_data_val, batch_size = 64, verbose=2), axis=-1)
y_pred_conf = model.predict(X_data_val, batch_size = 64, verbose=2)
y_test = np.nonzero(Y_data_val)[1]
#y_test = np.subtract(y_test, 1)
with open("test_result_dense.csv", 'w', encoding='UTF8', newline='') as tr:
    writer = csv.writer(tr)
    writer.writerow(["image_id", "pred_landmark", "true_landmark", "confidence_score"])
    for i in range(len(y_pred)):
        writer.writerow([y_test_ids[i], y_pred[i], y_test[i], np.max(y_pred_conf[i])])
            
print(y_pred)
print(y_test)

58/58 - 59s - 59s/epoch - 1s/step
58/58 - 38s - 38s/epoch - 653ms/step
[73 80 11 ... 33 37 52]
[73 80 11 ... 35 37 30]


In [6]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f'Model predication accuracy: {accuracy:.3f}')
print(f'\nClassification report:\n {classification_report(y_test, y_pred)}')

Model predication accuracy: 0.553

Classification report:
               precision    recall  f1-score   support

           0       0.43      0.56      0.49        16
           1       0.40      0.22      0.29        18
           2       0.44      0.39      0.41        18
           3       0.16      0.27      0.20        15
           4       0.54      0.41      0.47        17
           5       0.59      0.40      0.47        43
           6       0.80      0.95      0.87        21
           7       0.56      0.58      0.57        26
           8       0.48      0.56      0.52        57
           9       0.63      0.73      0.68        49
          10       0.60      0.67      0.63        45
          11       0.82      0.77      0.80       115
          12       0.71      0.75      0.73        20
          13       0.25      0.28      0.26        18
          14       0.56      0.60      0.58        15
          15       0.26      0.31      0.29        16
          16       0.2